In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-csv/train.csv

/kaggle/input/emsi-tabular-2024/sample_submission.csv

/kaggle/input/emsi-tabular-2024/train.csv

/kaggle/input/emsi-tabular-2024/test.csv


In [2]:
data = pd.read_csv("/kaggle/input/train-csv/train.csv")
#data_test = pd.read_csv("/kaggle/input/train-csv/train.csv")
sample_sub = pd.read_csv("/kaggle/input/emsi-tabular-2024/sample_submission.csv")
sample_sub

,id,smoking
0,159256,0.5
1,159257,0.5
2,159258,0.5
3,159259,0.5
4,159260,0.5
...,...,...
106166,265422,0.5
106167,265423,0.5
106168,265424,0.5
106169,265425,0.5


In [3]:
#remove duplicates
#data.drop_duplicates()
data

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,159251,40,155,45,69.0,1.5,2.0,1,1,127,...,72,159,14.5,1,0.8,25,26,13,0,0
159252,159252,50,155,75,82.0,1.0,1.0,1,1,120,...,64,108,14.5,1,0.6,21,20,18,0,0
159253,159253,40,160,50,66.0,1.5,1.0,1,1,114,...,87,93,10.9,1,0.6,15,9,12,0,0
159254,159254,50,165,75,92.0,1.2,1.0,1,1,121,...,55,80,14.4,1,1.1,22,17,37,0,1


Prediction target is y and X is the data we'll be using to predict smoking .**

In [4]:
y=data.smoking
X=data.drop(columns=['id','smoking'])


X

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,55,165,60,81.0,0.5,0.6,1,1,135,87,...,300,40,75,16.5,1,1.0,22,25,27,0
1,70,165,65,89.0,0.6,0.7,2,2,146,83,...,55,57,126,16.2,1,1.1,27,23,37,1
2,20,170,75,81.0,0.4,0.5,1,1,118,75,...,197,45,93,17.4,1,0.8,27,31,53,0
3,35,180,95,105.0,1.5,1.2,1,1,131,88,...,203,38,102,15.9,1,1.0,20,27,30,1
4,30,165,60,80.5,1.5,1.0,1,1,121,76,...,87,44,93,15.4,1,0.8,19,13,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,40,155,45,69.0,1.5,2.0,1,1,127,80,...,47,72,159,14.5,1,0.8,25,26,13,0
159252,50,155,75,82.0,1.0,1.0,1,1,120,80,...,202,64,108,14.5,1,0.6,21,20,18,0
159253,40,160,50,66.0,1.5,1.0,1,1,114,70,...,45,87,93,10.9,1,0.6,15,9,12,0
159254,50,165,75,92.0,1.2,1.0,1,1,121,90,...,148,55,80,14.4,1,1.1,22,17,37,0


In [5]:
xgb_final_params = {
    'learning_rate': 0.1, #taux d apprentissage : 
    'max_depth': 10, ##profondeur des arbres 
    'min_child_weight': 25,#fournit un seuil minimal quant au nombre d'individus présents dans un nœud.
    'n_estimators': 200, # le nombre d'arbres :un nombre plus grand permet d augmenter la performance mais 
    # augmenter le temps d'entrainement aussi.
    
    'objective': 'binary:logistic',
    #Binary classification refers to the task of classifying instances into one of two classes, 
    #often denoted as the positive class (1) and the negative class (0).
    #Logistic regression is a widely used algorithm for binary classification.
    #It models the probability of an instance belonging to the positive class using the logistic function (also known as the sigmoid function)
    
    'eval_metric': 'auc',#metrique utilisee area under the roc curve

    'seed': 42,#random number
    'verbosity': 1,
    'n_jobs': -1#use all available processes.
}

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#Spliting data
train_X, val_X, train_y, val_y = train_test_split(X, y,test_size=0.666666,random_state = 42)

#This dataset uses the Korean vision scoring system which goes from 0.1 to 2.0. 
#In this dataset an eyesight value of 9.9 indicates blindness. In order to keep ordinality we will change this so 0 indicates blindness

train_X['eyesight(left)'] = train_X['eyesight(left)'].replace(to_replace=9.9, value=0)
train_X['eyesight(right)'] = train_X['eyesight(right)'].replace(to_replace=9.9, value=0)

val_X['eyesight(left)'] = val_X['eyesight(left)'].replace(to_replace=9.9, value=0)
val_X['eyesight(right)'] = val_X['eyesight(right)'].replace(to_replace=9.9, value=0)

#Using Xgboost
# Create an XGBoost classifier
model = XGBClassifier(**xgb_final_params)

# Fit the model on the training data
model.fit(train_X,train_y)




XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=25, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

In [8]:
#Prepare the test_data
data_test= pd.read_csv("/kaggle/input/emsi-tabular-2024/test.csv")
data_test = data_test.drop(columns=['id'])
data_test['eyesight(left)'] = data_test['eyesight(left)'].replace(to_replace=9.9, value=0)
data_test['eyesight(right)'] = data_test['eyesight(right)'].replace(to_replace=9.9, value=0)


data_test

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,40,165,70,84.0,1.2,1.2,1,1,130,89,...,186,49,115,14.2,1,0.9,19,25,32,0
1,80,160,60,93.0,1.0,1.0,2,2,144,72,...,158,35,104,13.0,1,1.1,20,12,24,0
2,60,170,70,86.5,0.6,0.7,1,1,117,75,...,173,39,88,15.4,1,1.4,38,60,36,0
3,40,160,50,67.0,0.3,0.4,1,1,116,62,...,47,75,128,14.5,1,0.6,25,18,10,1
4,40,170,75,89.4,1.0,0.9,1,1,132,94,...,100,39,123,16.5,1,1.0,30,39,27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106166,40,165,60,78.0,0.8,0.9,1,1,112,78,...,82,59,149,16.4,1,1.1,24,31,28,0
106167,40,170,85,95.0,1.2,1.2,1,1,130,90,...,186,44,100,16.0,2,1.0,25,31,38,0
106168,35,170,85,89.0,1.2,1.2,1,1,131,86,...,76,53,142,15.9,1,0.8,33,32,24,1
106169,40,160,60,67.0,0.7,0.8,1,1,120,80,...,81,55,103,13.9,1,0.6,42,36,32,0


In [9]:
#create the model
data1=model.predict_proba(data_test)[:, 1]
#This slicing operation is used to extract the predicted probabilities for the positive class


In [10]:
data1

array([0.5849677 , 0.26747817, 0.37165946, ..., 0.54246855, 0.05742507,
       0.08538195], dtype=float32)

In [11]:
sample_sub['smoking'] = data1
sample_sub

,id,smoking
0,159256,0.584968
1,159257,0.267478
2,159258,0.371659
3,159259,0.009673
4,159260,0.627510
...,...,...
106166,265422,0.638197
106167,265423,0.595910
106168,265424,0.542469
106169,265425,0.057425


In [12]:
sample_sub.to_csv('submission.csv',index = False)
pd.read_csv('submission.csv')

,id,smoking
0,159256,0.584968
1,159257,0.267478
2,159258,0.371659
3,159259,0.009673
4,159260,0.627510
...,...,...
106166,265422,0.638197
106167,265423,0.595911
106168,265424,0.542469
106169,265425,0.057425
